# Initialization

## Import Trajectories

In [ ]:
from os import listdir
from os.path import isfile, join
dataset_path = 'converted_20_traj/'
onlyfiles = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

import numpy as np
traj_files = [dataset_path + f for f in sorted(onlyfiles)]
dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe]

dataset_path2 = 'converted_100_traj/'
onlyfiles2 = [f for f in listdir(dataset_path2) if isfile(join(dataset_path2, f))]
traj_files2 = [dataset_path2 + f for f in sorted(onlyfiles2)]
dataframe2 = [np.loadtxt(traj) for traj in traj_files2]
series2 = [serie.astype('float32') for serie in dataframe2]

## Transformation Matrix Function Definition

In [ ]:
from math import *

def calculateRT(xteta,yteta,zteta,xtr,ytr,ztr):

    R1=np.array([[cos(zteta), -sin(zteta), 0 ],
        [sin(zteta),  cos(zteta), 0 ],
        [0,           0,          1 ]])
    R2=np.array([[cos(yteta),  0, sin(yteta) ],
        [0,           1, 0          ],
        [-sin(yteta), 0, cos(yteta) ]] )   
    R3=np.array([[1,           0,           0          ],
        [0,           cos(xteta), -sin(xteta) ],
        [0,           sin(xteta),  cos(xteta) ]])
    R=np.dot(np.dot(R1,R2),R3)
    H=np.zeros((4,4))
    H[0:3,0:3]=R;
    H[0,3]=xtr
    H[1,3]=ytr
    H[2,3]=ztr
    H[3,3]=1
    return H


## Calculate Trajectories and Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
orij_plot_list = list()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(0, 0.15)
r = 0.1
x = np.linspace(r, -r, 100)
z = np.linspace(-0.00, 0.005, 100)
Xc, Zc = np.meshgrid(x, z)
Yc = np.sqrt(r**2 - Xc**2)

# Draw parameters
rstride = 200
cstride = 1
ax.plot_surface(
    Xc, Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride, color='yellow')
ax.hold(True)
ax.plot_surface(
    Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride, color='yellow')
alltraj2 = list()
count = 1
for serie in series2:
    count = count + 1
    orijData = copy.copy(serie)
    orijTraj = np.zeros((11, 3))
    #    print orijData
    orijBaseFrame = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0],
                              [0, 0, 0, 1]])
    for i in range(1, 10):
        H = calculateRT(orijData[i, 3], orijData[i, 4], orijData[i, 5],
                        orijData[i, 0], orijData[i, 1], orijData[i, 2])
        orijBaseFrame = np.dot(H, orijBaseFrame)
        orijTraj[i + 1, :] = orijBaseFrame[0:3, 3]
    ax.plot(orijTraj[:, 0], orijTraj[:, 1], orijTraj[:, 2])
    alltraj2.append(orijTraj)
    orij_plot_list.append(
        np.sqrt(orijTraj[:, 0]**2 + orijTraj[:, 1]**2 + orijTraj[:, 2]**2))

plt.show()

# First Step LSTM

## LSTM 1 Train

In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras import optimizers
from keras.layers import TimeDistributed
from math import sqrt
import matplotlib
import numpy
import numpy as np
import copy
import random
from numpy import concatenate
from mpl_toolkits.mplot3d import Axes3D
# frame a sequence as a supervised learning problem
lstm_model1 = None
scaler = None


def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag + 1)]
    columns.append(df)
    df = concat(columns, axis=1)
    return df


# fit an LSTM network to training data
def fit_lstm(train, batch_size, neurons):
    X, y = train[:, :8], train[:, 8:14]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(
        LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(6), )
    model.compile(loss='mean_squared_error', optimizer="adaDelta")
    for i in range(1):
        model.fit(
            X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model


# run a repeated experiment
def experiment(series):
    # transform data to be stationary
    global scaler
    diff_set = list()
    seriesData = numpy.array(copy.copy(series))
    scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(
        seriesData.reshape(seriesData.shape[0] * seriesData.shape[1],
                           seriesData.shape[2]))
    global lstm_model1
    for k in range(80):
        print k
        count = 0
        #if k>0:
        #    print lstm_model1.optimizer.lr.set_value(1/(k+1))
        a = range(75)
        random.shuffle(a)
        for i in a:
            serie = series[i]
            count = count + 1
            raw_values = scaler.transform(copy.copy(serie))
            # transform data to be supervised learning
            supervised = timeseries_to_supervised(raw_values, 1)
            supervised_values = supervised.values[1:10, :]
            supervised_values_x = supervised_values[:, 0:8]
            supervised_values_y = supervised_values[:, 8:]
            # split data into train and test-sets
            scaled_supervised_values_x = supervised_values_x 
            scaled_supervised_values_y = supervised_values_y 
            dataset = np.concatenate(
                (scaled_supervised_values_x, scaled_supervised_values_y),
                axis=1)
            # run experiment
            # fit the base model
            #if count in TrainIndexList:
            if lstm_model1 == None:
                lstm_model1 = fit_lstm(dataset, 9, 50)
            else:
                X, y = dataset[:, :8], dataset[:, 8:14]
                X = X.reshape(X.shape[0], 1, X.shape[1])
                for i in range(1):
                    lstm_model1.fit(
                        X, y, epochs=1, batch_size=9, verbose=1, shuffle=False)
                    lstm_model1.reset_states()


# execute the experiment
def run():
    # experiment
    results = DataFrame()
    # run experiment
    results['results'] = experiment(series2)
    # summarize results
    print(results.describe())
    # save results
    results.to_csv('experiment_stateful.csv', index=False)

# entry point
run()

## 1NN and Random Trajectory Prediction

In [ ]:
import random
diff_plot_list_baseline1 = list()
diff_plot_list_baseline2 = list()
for i in range(75, 100):
    print i
    lever_up_position = series2[i][0, 6:8]
    smallest = 1000
    index = 0
    for j in range(75):
        diff = lever_up_position - series2[j][0, 6:8]
        diffT = diff[0]**2 + diff[1]**2
        if diffT < smallest:
            smallest = diffT
            index = j
    randIndex = random.randint(0, 75)
    originalTraj = alltraj2[i]
    predictedTraj1 = alltraj2[index]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    r=0.1;
    x=np.linspace(r, -r, 100)
    z=np.linspace(-0.00, 0.005, 100)
    Xc, Zc=np.meshgrid(x, z)
    Yc = np.sqrt(r**2-Xc**2)

    # Draw parameters
    rstride = 200
    cstride = 1
    ax.plot_surface(Xc, Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
    ax.plot_surface(Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
    ax.scatter(series2[i][0, 6], series2[i][0, 7])
    ax.plot(
        originalTraj[:, 0],
        originalTraj[:, 1],
        originalTraj[:, 2],
        color='green')
    ax.plot(
        predictedTraj1[:, 0],
        predictedTraj1[:, 1],
        predictedTraj1[:, 2],
        color='red')
    plt.show()

    diffTraj1 = originalTraj - predictedTraj1
    diff_plot_list_baseline1.append(
        np.sqrt(diffTraj1[:, 0]**2 + diffTraj1[:, 1]**2 + diffTraj1[:, 2]**2))
    predictedTraj2 = alltraj2[randIndex]
    diffTraj2 = originalTraj - predictedTraj2
    diff_plot_list_baseline2.append(
        np.sqrt(diffTraj2[:, 0]**2 + diffTraj2[:, 1]**2 + diffTraj2[:, 2]**2))

## LSTM trajectory Prediction

In [ ]:
diff_plot_list=list()
orij_plot_list=list()
test_plot_list=list()
orj_Translation_list=list()
test_Translation_list=list()
diff_Translation_list=list()
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm1_model1
    global scaler
    for index in range(75,100):
        print index
        orijData=series[index]
        scaled_serie=scaler.transform(orijData)
        Xhat=scaled_serie[:-1,0:8]

        X = np.zeros((9, 1, 8))
        Xk=Xhat[0,:]
        for i in range(9):
            X[i,:]=Xk.reshape(1, 1, 8)
        for i in range(8):
            
            #X = Xhat.reshape(13, 1, 10)
            #print X
            yhat = lstm_model1.predict(X, batch_size=9)
            Xk=yhat[i,:]
            X[i+1,0,0:6]=Xk.reshape(1, 1, 6)
            lstm_model1.reset_states()
        testData =scaler.inverse_transform(X.reshape(9,8))
        testTraj=np.zeros((10,3))
        orijTraj=np.zeros((10,3))
        testBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(9): 
            H=calculateRT(testData[i,3],testData[i,4],testData[i,5],testData[i,0],testData[i,1],testData[i,2]);
            testBaseFrame=np.dot(H,testBaseFrame);
            testTraj[i+1,:]=testBaseFrame[0:3,3];
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];
            
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #ax.set_zlim(0, 0.15)
        #ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(orijData[0,6],orijData[0,7])
        diffTraj=orijTraj-testTraj
        diff_plot_list.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
        orij_plot_list.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
        test_plot_list.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        #diffData=orijData[1:,:]-testData
        #orj_Translation_list.append(np.sqrt(orijData[:,0]**2+orijData[:,1]**2+orijData[:,2]**2))
        #test_Translation_list.append(np.sqrt(testData[:,0]**2+testData[:,1]**2+testData[:,2]**2))
        #diff_Translation_list.append(np.sqrt(diffData[:,0]**2+diffData[:,1]**2+diffData[:,2]**2))
        
        ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='green')
        ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red')
        r=0.1;
        x=np.linspace(r, -r, 100)
        z=np.linspace(-0.00, 0.005, 100)
        Xc, Zc=np.meshgrid(x, z)
        Yc = np.sqrt(r**2-Xc**2)

        # Draw parameters
        rstride = 200
        cstride = 1
        ax.plot_surface(Xc, Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        ax.plot_surface(Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        plt.show()
        lstm_model1.reset_states()

#dataframe = [np.loadtxt(traj) for traj in traj_files]
#series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series2)

## LSTM Plot Prediction Knowing few step of trajecory

In [ ]:
diff_plot_list=list()
orij_plot_list=list()
test_plot_list=list()
orj_Translation_list=list()
test_Translation_list=list()
diff_Translation_list=list()
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm1_model1
    global scaler
    for index in range(75,100):
        orijData=series[index]
        scaled_serie=scaler.transform(orijData)
        Xhat=scaled_serie[:-1,0:8]

        X = np.zeros((9, 1, 8))
        Xk=Xhat[0:5,:]
        X[0:5,:]=Xk.reshape(5, 1, 8)
        for i in range(5,9):
            X[i,:]=Xk[-1,:].reshape(1, 1, 8)
        for i in range(4,8):
            
            #X = Xhat.reshape(13, 1, 10)
            #print X
            yhat = lstm_model1.predict(X, batch_size=9)
            Xk=yhat[i,:]
            X[i+1,0,0:6]=Xk.reshape(1, 1, 6)
            lstm_model1.reset_states()
        testData =scaler.inverse_transform(X.reshape(9,8))
        testTraj=np.zeros((10,3))
        orijTraj=np.zeros((10,3))
        testBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(9): 
            H=calculateRT(testData[i,3],testData[i,4],testData[i,5],testData[i,0],testData[i,1],testData[i,2]);
            testBaseFrame=np.dot(H,testBaseFrame);
            testTraj[i+1,:]=testBaseFrame[0:3,3];
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];
            
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #ax.set_zlim(0, 0.15)
        #ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(orijData[0,6],orijData[0,7])
        diffTraj=orijTraj-testTraj
        diff_plot_list.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
        orij_plot_list.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
        test_plot_list.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        #diffData=orijData[1:,:]-testData
        #orj_Translation_list.append(np.sqrt(orijData[:,0]**2+orijData[:,1]**2+orijData[:,2]**2))
        #test_Translation_list.append(np.sqrt(testData[:,0]**2+testData[:,1]**2+testData[:,2]**2))
        #diff_Translation_list.append(np.sqrt(diffData[:,0]**2+diffData[:,1]**2+diffData[:,2]**2))
        
        ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='green')
        ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red')
        r=0.1;
        x=np.linspace(r, -r, 100)
        z=np.linspace(-0.00, 0.005, 100)
        Xc, Zc=np.meshgrid(x, z)
        Yc = np.sqrt(r**2-Xc**2)

        # Draw parameters
        rstride = 200
        cstride = 1
        ax.plot_surface(Xc, Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        ax.plot_surface(Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        plt.show()
        lstm_model1.reset_states()

#dataframe = [np.loadtxt(traj) for traj in traj_files]
#series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series2)

# Error Plots

In [ ]:
asd=np.mean(np.array(diff_plot_list_baseline1),axis=0)
asdvar=np.std(np.array(diff_plot_list_baseline1),axis=0)
asd2=np.mean(np.array(diff_plot_list_baseline2),axis=0)
asdvar2=np.std(np.array(diff_plot_list_baseline2),axis=0)
asd3=np.mean(np.array(diff_plot_list),axis=0)[0:10]
asdvar3=np.std(np.array(diff_plot_list7),axis=0)[0:10]
x=np.array([i for i in range(0,11)])
baseline1=plt.plot(asd,color='red',label='Low Baseline - 1NN')
plt.hold(True)
plt.fill_between(x,asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
baseline2=plt.plot(asd2,color='blue',label='Hard Baseline - Random Trajectory Selection')
plt.hold(True)
plt.fill_between(x,asd2+asdvar2,asd2-asdvar2,alpha=0.3,color='lightblue')
plt.hold(True)
baseline3=plt.plot(x[1:11],asd3,color='green',label='LSTM')
plt.hold(True)
plt.fill_between(x[1:11],asd3+asdvar3,asd3-asdvar3,alpha=0.3,color='lightgreen')
plt.hold(True)
axes = plt.gca()
axes.set_ylim([0,0.15])
axes.set_xlim([0,10])
plt.legend( ['Low Baseline - 1NN', 'Hard Baseline - Random Trajectory Selection','LSTM'])
#plt.legend(handles=[baseline1, baseline2])
#plt.plot(asdasd,color='black')
plt.show()

In [ ]:
asd=np.mean(np.array(diff_plot_list_baseline2),axis=0)
asdvar=np.std(np.array(diff_plot_list_baseline2),axis=0)
asd2=np.mean(np.array(diff_plot_list1),axis=0)[0:10]
asdvar2=np.std(np.array(diff_plot_list1),axis=0)[0:10]
asd3=np.mean(np.array(diff_plot_list3),axis=0)[0:10]
asdvar3=np.std(np.array(diff_plot_list3),axis=0)[0:10]
asd4=np.mean(np.array(diff_plot_list5),axis=0)[0:10]
asdvar4=np.std(np.array(diff_plot_list5),axis=0)[0:10]
asd5=np.mean(np.array(diff_plot_list7),axis=0)[0:10]
asdvar5=np.std(np.array(diff_plot_list7),axis=0)[0:10]
x=np.array([i for i in range(0,11)])
baseline1=plt.plot(asd,color='red',label='rastgele')
plt.hold(True)
plt.fill_between(x,asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
baseline2=plt.plot(x[1:11],asd2,color='orange',label='1')
plt.hold(True)
plt.fill_between(x[1:11],asd2+asdvar2,asd2-asdvar2,alpha=0.4,color='orange')
plt.hold(True)
baseline3=plt.plot(x[1:11],asd3,color='yellow',label='3')
plt.hold(True)
plt.fill_between(x[1:11],asd3+asdvar3,asd3-asdvar3,alpha=0.8,color='lightyellow')
plt.hold(True)
baseline4=plt.plot(x[1:11],asd4,color='blue',label='5')
plt.hold(True)
plt.fill_between(x[1:11],asd4+asdvar4,asd4-asdvar4,alpha=0.6,color='lightblue')
plt.hold(True)
baseline5=plt.plot(x[1:11],asd5,color='green',label='7')
plt.hold(True)
plt.fill_between(x[1:11],asd5+asdvar5,asd5-asdvar5,alpha=0.4,color='lightgreen')
plt.hold(True)
axes = plt.gca()
axes.set_ylim([0,0.15])
axes.set_xlim([0,10])
plt.legend( ['Random Selection', 
             'Using first point only',
             'Using first 3 point only',
             'Using first 5 point only',
             'Using first 7 point only'])
plt.show()

# Second LSTM

## LSTM Model 2 Train

In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import matplotlib
import numpy
import numpy as np
from numpy import concatenate
from mpl_toolkits.mplot3d import Axes3D
# frame a sequence as a supervised learning problem
lstm_model2 = None


def fit_lstm(train, batch_size, neurons):
    X, y = train[:, 0:6], train[:, 6:8]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(
        LSTM(
            neurons,
            batch_input_shape=(batch_size, X.shape[1], X.shape[2]),
            stateful=True))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.compile(loss='mean_squared_error', optimizer='Adagrad')
    for i in range(5):
        model.fit(
            X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model


def experiment(series):
    # transform data to be stationary
    global lstm_model2
    for i in range(40):
        a = range(75)
        random.shuffle(a)
        for j in a:
            serie = series[j]
            raw_values = scaler.transform(serie)
            # transform data to be supervised learning
            raw_values = raw_values[:-2, :]
            scaled_supervised_values_x = raw_values[:, 0:6]
            scaled_supervised_values_y = raw_values[:, 6:8]
            dataset = np.concatenate(
                (scaled_supervised_values_x, scaled_supervised_values_y),
                axis=1)
            if lstm_model2 == None:
                lstm_model2 = fit_lstm(dataset, 9, 50)
            else:
                X, y = dataset[:, 0:6], dataset[:, 6:8]
                X = X.reshape(X.shape[0], 1, X.shape[1])
                lstm_model2.fit(
                    X, y, epochs=1, batch_size=9, verbose=1, shuffle=False)
                lstm_model2.reset_states()


def run():
    results = DataFrame()
    # run experiment
    results['results'] = experiment(series2)
    # summarize results
    print(results.describe())
    # save results
    results.to_csv('experiment_stateful.csv', index=False)


# entry point
run()

## LSTM Model 2 Prediction

In [ ]:
#pointPredictionList=list()
pointPredictionList2=list()
testData=0
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm_model2
    global scaler 
    global testData
    global serie
    for i in range(75,100):
        serie=series[i]
        scaled_serie=scaler.transform(serie)
        scaled_serie=scaled_serie[:-2,:]
        X = scaled_serie.reshape(9, 1, 8)
        X = X[:,:,0:6]
        y = lstm_model2.predict(X, batch_size=9)
        lstm_model2.reset_states()
        testData=np.concatenate((X.reshape(9,6),y),1)
        testData =scaler.inverse_transform(testData)        #Xhat=yhat
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        r=0.1;
        x=np.linspace(r, -r, 100)
        z=np.linspace(-0.00, -0.005, 100)
        Xc, Zc=np.meshgrid(x, z)
        Yc = np.sqrt(r**2-Xc**2)

        # Draw parameters
        rstride = 200
        cstride = 1
        ax.plot_surface(Xc, Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        ax.hold(True)
        angles= np.arctan2(testData[:,7], testData[:,6]) 
        testData[:,7]=0.1*np.sin(angles)
        testData[:,6]=0.1*np.cos(angles)
        ax.plot_surface(Xc, -Yc, Zc, alpha=0.2, rstride=rstride, cstride=cstride,color='yellow')
        ax.plot(alltraj2[i][:-1,0],alltraj2[i][:-1,1],alltraj2[i][:-1,2],color='green')
        diff=serie[:-2,6:8]-testData[:,6:8]
        pointPredictionList2.append(np.sqrt(diff[:,0]**2+diff[:,1]**2))
        ax.scatter(serie[0,6],serie[0,7],0, c='blue', edgecolors='blue',label='Gercek')
        ax.hold(True)
        ax.scatter(testData[0:,6],testData[0:,7],0, edgecolors='r', c='r')
        

        #plt.legend( ['Gercek','Tahmin Edilen'])
        #print np.concatenate((-testData[:,6],testData[:,7]))-np.matlib.repmat([-serie[0,6],serie[0,7]],1,29)
        plt.show()
        lstm_model2.reset_states()
        
#dataframe = [np.loadtxt(traj) for traj in traj_files]
#series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series2)

## LSTM Model 2 Error

In [ ]:
asd=np.mean(np.array(pointPredictionList),axis=0)
asdvar=np.std(np.array(pointPredictionList),axis=0)
asd2=np.mean(np.array(pointPredictionList2),axis=0)
asdvar2=np.std(np.array(pointPredictionList2),axis=0)
plt.plot(x[1:-1],asd,color='red',label='oncesi')
plt.hold(True)
plt.fill_between(x[1:-1],asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
plt.plot(x[1:-1],asd2,color='blue',label='sonrasi')
plt.hold(True)
plt.fill_between(x[1:-1],asd2+asdvar2,asd2-asdvar2,alpha=0.3,color='lightblue')
plt.hold(True)
plt.legend( ['ileri isleme oncesi','ileri isleme sonrasi'])

plt.show()

# Models Mixed

In [ ]:
diff_plot_list1=list()
orij_plot_list1=list()
test_plot_list1=list()
diff_plot_list3=list()
orij_plot_list3=list()
test_plot_list3=list()
diff_plot_list5=list()
orij_plot_list5=list()
test_plot_list5=list()
diff_plot_list7=list()
orij_plot_list7=list()
test_plot_list7=list()
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm_model
    global lstm_model2
    global scaler 
    for i in range(75,100):
        orijData=series[i]
        scaled_serie=scaler.transform(series[i])
        scaled_serie=scaled_serie[:-2,:]
        X = scaled_serie.reshape(9, 1, 8)
        X = X[:,:,0:6]
        yks=np.zeros((9,1,2));        
        testPlots=list();
        for i in range(1,9):
            Xcur=np.zeros((9,1,6))
            Xcur[:i,:,:]=X[:i,:,:]
            for j in range(i,9):
                Xcur[j,:,:]=Xcur[i-1,:,:]
            y = lstm_model2.predict(Xcur, batch_size=9)
            lstm_model2.reset_states()
            testData=np.concatenate((X.reshape(9,6),y),1)
            testData =scaler.inverse_transform(testData)        #Xhat=yhat
            angles= np.arctan2(testData[:,7], testData[:,6]) 
            testData[:,7]=0.099*np.sin(angles)
            testData[:,6]=0.099*np.cos(angles)
            testData2=scaler.transform(testData)
            yks[i-1,:,:]=y[i-1,:].reshape((1,1,2))
            X2cur=np.zeros((9,1,8))   
            X2cur[:,:,:6]=Xcur
            for j in range(0,9):
                X2cur[j,:,6:]=testData2[i-1,6:8];
            for j in range(i,9):
                yhat = lstm_model1.predict(X2cur, batch_size=9)
                Xhat=yhat
                Xk=Xhat[j-1,:]
                X2cur[j,0,0:6]=Xk.reshape(1, 1, 6)
                lstm_model1.reset_states()            
            testPlots.append(X2cur)
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(orijData[0,6],orijData[0,7],0,color='blue')
        
        orijTraj=np.zeros((10,3))
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(9): 
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];        
        
        ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='blue')
        predictedNextPos=np.zeros((9,3));
        for i in range(0,8):
            testData =scaler.inverse_transform(testPlots[i].reshape((9,8)))
            testBaseFrame=[[1, 0, 0, 0],
                           [0, 1, 0, 0],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]]
            testTraj=np.zeros((10,3))
            for j in range(9): 
                H=calculateRT(testData[j,3],testData[j,4],testData[j,5],testData[j,0],testData[j,1],testData[j,2]);
                testBaseFrame=np.dot(H,testBaseFrame);
                testTraj[j+1,:]=testBaseFrame[0:3,3];
            predictedNextPos[i+1,:]=testTraj[i,0:3]
            #ax.scatter(serie[i+1,0],serie[i+1,1],serie[i+1,2],c='#0000FF')
            #ax.scatter(testData[i,0],testData[i,1],testData[i,2],c='#00FF00')
            diffTraj=orijTraj[:,:3]-testTraj[:,:3]
            if(i==0):
                diff_plot_list1.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list1.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list1.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
                
            elif(i==2):
                diff_plot_list3.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list3.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list3.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))

            elif(i==4):
                diff_plot_list5.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list5.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list5.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
                ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red'); 

            elif(i==6):
                diff_plot_list7.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list7.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list7.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        #ax.plot(predictedNextPos[:,0],predictedNextPos[:,1],predictedNextPos[:,2],color='green')
        plt.show()
        
        
#dataframe = [np.loadtxt(traj) for traj in traj_files]
#series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series2)